In [ ]:
import matplotlib.pyplot as plt
from citkid.prima_readout.generator import generate_data
from citkid.prima_readout.create_cal import make_x_cal
from citkid.prima_readout.apply_cal import convert_iq_to_x
from citkid.prima_readout.data_io import *
from citkid.res.gain import remove_gain
from citkid.prima_readout.cosmic_rays import remove_cosmic_rays
from citkid.prima_readout.average import average_x

## Generate random data 
The following function generates fine scan data (ffine, zfine), gain scan data (fgain, zgain), and a timestream (f0 is the tone frequency and znoise is the timestream). The fine and gain scans have random parameters, and the noise timestream has fixed parameters (I want to vary these eventually), but scaled to the IQ loop. noise_factor is used to increase or decrease the average amount of noise in the IQ loops. 

In [ ]:
ffine, zfine, fgain, zgain, f0, znoise = generate_data(noise_factor = 0)

## Create and save calibration file
The following code is run on the ground to create calibration files, given the data that is measured (or in this case generated).

In [ ]:
# Create calibration data and plots
fs = [np.mean(ffine)]
Qs = [10000] # This scales with resonator Q
p_amp, p_phase, origin, v, theta_fine, p_x, figs =\
make_x_cal(ffine, zfine, fgain, zgain, fs, Qs, plotq = True)

# Save data and plots
data_directory = 'C:/data/citkid_dev/prima_readout/test/'
plot_directory = data_directory + 'plots/'
filename = 'test'
save_x_cal(data_directory, filename, p_amp, p_phase, origin, v, p_x, figs, 
           plot_directory = plot_directory, make_directories = True)
ax = figs[1].axes[0]

## Load calibration file

In [ ]:
p_amp, p_phase, origin, v, p_x = load_x_cal(data_directory + filename + '.npz')

## Convert znoise to x using the loaded calibration

In [ ]:
x = convert_iq_to_x(f0, znoise, p_amp, p_phase, origin, v, p_x)

## Remove cosmic rays
We would characterize the time constant of each detector ahead of time. We have to characterize the spread accross the array: maybe this can be a fixed value for the whole array. cr_nstd will also be characterized ahead of time. I suspect it will depend on the noise of each detector. We need to do some work on cosmic ray generator, because they look a little weird right now. There is also some work to do on the peak finding algorithm parameters.

In [ ]:
fsample = 10e3 # 10 kHz 
tsample = 1 / fsample
cr_indices, x_rmvd = remove_cosmic_rays(x, tsample, cr_nstd = 6, time_constant = 1e-3)

## Average data to 500 Hz, and write to a file

In [ ]:
navg = 10000 // 500
x_avg = average_x(x_rmvd, navg)

In [ ]:
np.save(data_directory + 'xavg.npy', x_avg)

# Plotting some of the data for reference

## Raw IQ and timestream
Here is the raw fine and gain scan sweep data plotted in $|S_{21}|$ versus frequency space and IQ space, with the timestream plotted on the IQ plot.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = [12, 4], dpi = 300, layout = 'tight')
axs[0].set(ylabel = r'$|S_{21}|$ (dB)', xlabel = 'Frequency (GHz)')
axs[1].set(ylabel = 'Q', xlabel = 'I')
axs[1].set_aspect('equal')

axs[0].plot(fgain / 1e9, 20 * np.log10(abs(zgain)), '.', color = plt.cm.viridis(0.1))
axs[0].plot(ffine / 1e9, 20 * np.log10(abs(zfine)), '.', color = plt.cm.viridis(0.9))

axs[1].plot(np.real(zgain), np.imag(zgain), '.', color = plt.cm.viridis(0.1), label = 'gain scan')
axs[1].plot(np.real(zfine), np.imag(zfine), '.', color = plt.cm.viridis(0.9), label = 'fine scan')

axs[1].plot(np.real(znoise[::10]), np.imag(znoise[::10]), '.', color = plt.cm.Greys(0.5), label = 'timestream')
axs[1].legend()

## Remove gain from S21 data, and center the circle on 0 + 0j
This is the result of the first part of the the conversion from IQ to x. I have applied the conversion to the fine sweep data and the timestream. On the IQ plot, I have also plotted v, the vector that defines the angle at which $\theta=0$.

In [ ]:
from citkid.res.gain import remove_gain
zfine_rmvd = remove_gain(ffine, zfine, p_amp, p_phase)
zfine_shifted = zfine_rmvd - origin

znoise_rmvd = remove_gain(f0, znoise, p_amp, p_phase)
znoise_shifted = znoise_rmvd - origin

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = [12, 4], dpi = 300, layout = 'tight')
axs[0].set(ylabel = r'$|S_{21}|$ (dB)', xlabel = 'Frequency (GHz)')
axs[1].set(ylabel = 'Q', xlabel = 'I')
axs[1].set_aspect('equal')

axs[0].plot(ffine / 1e9, 20 * np.log10(abs(zfine_shifted)), '.', color = plt.cm.viridis(0.9))

r0 = abs(np.mean(znoise_shifted)) # amplitude for plotting, but only the angle of v is relevant for analysis
axs[1].plot([0, np.real(v) * r0], [0, np.imag(v) * r0], '-k', label = r'v ($\theta = 0$)')
axs[1].plot(np.real(zfine_shifted), np.imag(zfine_shifted), '.', color = plt.cm.viridis(0.9), label = 'fine scan')
axs[1].plot(np.real(znoise_shifted[::10]), np.imag(znoise_shifted[::10]), '.', color = plt.cm.Greys(0.5), label = 'timestream')

axs[1].legend()

## Cosmic ray removal and averaging
Here is a timestream before and after cosmic ray removal. We need to fine tune a few things here, so the cosmic ray removal is not perfect right now.

In [ ]:
# Plot 
fig, ax = plt.subplots(figsize = [6, 2], dpi = 150, layout = 'tight')
ax.set(xlabel = 'Time (s)', ylabel = 'x (Hz / MHz)')
t = np.arange(0, tsample * len(x), tsample)
ax.plot(t, x, color = plt.cm.Greys(0.5))
ax.plot(t[cr_indices], x[cr_indices], 'xr')
ax.plot(t, x_rmvd, color = 'b')
t_avg = np.arange(0, len(x_avg) * tsample * navg, tsample * navg)
ax.plot(t_avg, x_avg, color = 'r')

## PSDs
Here are the corresponding PSDs of the x timestreams with and without cosmic ray removal.

In [ ]:
from citkid.noise.psd import bin_psd, get_psd
x = convert_iq_to_x(f0, znoise, p_amp, p_phase, origin, v, p_x)
f, sxx = get_psd(x, tsample, get_frequencies = True)
fbin, sxxbin = bin_psd(f, [f, sxx], fmin = 0.1, nbins = 80)
f_rmvd, sxx_rmvd = get_psd(x_rmvd, tsample, get_frequencies = True)
fbin_rmvd, sxxbin_rmvd = bin_psd(f_rmvd, [f_rmvd, sxx_rmvd], fmin = 0.1, nbins = 80)

In [ ]:
fig, ax = plt.subplots(figsize = [4, 3], dpi = 300)
ax.set(yscale = 'log', xscale = 'log', ylabel = r'$S_{xx}$ (1 / Hz)', xlabel = 'Frequency (Hz )')
ax.plot(fbin, sxxbin, color = plt.cm.viridis(0.1), label = 'with cosmic rays')
ax.plot(fbin_rmvd, sxxbin_rmvd, color = plt.cm.viridis(0.9), label = 'cosmic rays removed')
ax.legend()